In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

# ULMFiT

In [2]:
from fastai.text.all import *

<IPython.core.display.Javascript object>

## Language model

In [3]:
path = Path("../input")

<IPython.core.display.Javascript object>

In [4]:
df_train = pd.read_csv(path / "train.csv")
df_train.columns = ["index", "text", "label"]

df_test = pd.read_csv(path / "test_x.csv")
df_test["author"] = -1
df_test.columns = ["index", "text", "label"]

df_all = pd.concat([df_train, df_test])
len(df_train), len(df_test), len(df_all)

(54879, 19617, 74496)

<IPython.core.display.Javascript object>

In [5]:
data_lm = DataBlock(
    blocks=TextBlock.from_df("text", is_lm=True),
    get_x=ColReader("text"),
    splitter=RandomSplitter(valid_pct=0.1),
)

<IPython.core.display.Javascript object>

In [6]:
bs = 196

<IPython.core.display.Javascript object>

In [7]:
dls_lm = data_lm.dataloaders(df_all, bs=bs)
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos “ no , indeed , you know your duty too well for me to -- even xxunk ” xxbos xxmaj and at last his patience was rewarded . xxmaj it was a fine dry night ; frost in the air ; the streets as clean as a ballroom floor ; the lamps , unshaken by any wind , drawing a regular pattern of light and shadow . xxmaj by ten o’clock","“ no , indeed , you know your duty too well for me to -- even xxunk ” xxbos xxmaj and at last his patience was rewarded . xxmaj it was a fine dry night ; frost in the air ; the streets as clean as a ballroom floor ; the lamps , unshaken by any wind , drawing a regular pattern of light and shadow . xxmaj by ten o’clock ,"
1,"a hansom cab drove up to odin xxmaj lodge , and a gentleman sprang out . xxmaj he was a remarkably handsome man , dark , aquiline , and moustached — evidently the man of whom i had heard . xxmaj he appeared to be in a great hurry , shouted to the cabman to wait , and brushed past the maid who opened the door with the air of a man","hansom cab drove up to odin xxmaj lodge , and a gentleman sprang out . xxmaj he was a remarkably handsome man , dark , aquiline , and moustached — evidently the man of whom i had heard . xxmaj he appeared to be in a great hurry , shouted to the cabman to wait , and brushed past the maid who opened the door with the air of a man who"


<IPython.core.display.Javascript object>

In [8]:
opt_func = partial(Adam, wd=0.1)
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    metrics=[accuracy, Perplexity()],
    path=path,
    #     pretrained=False,
    drop_mult=0.5,
)
learn = learn.to_fp16(clip=0.1)

<IPython.core.display.Javascript object>

In [9]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.049352,3.757093,0.281809,42.823776,01:17


<IPython.core.display.Javascript object>

In [10]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.818186,3.645567,0.296258,38.304474,01:21
1,3.719707,3.563044,0.307150,35.270401,01:21
2,3.600868,3.472086,0.318494,32.203850,01:21
3,3.465043,3.399265,0.328937,29.942080,01:22
4,3.349673,3.345675,0.336981,28.379725,01:22
5,3.240811,3.302299,0.344853,27.175043,01:22
6,3.147867,3.274128,0.350679,26.420176,01:22
7,3.080234,3.255558,0.354823,25.934082,01:22
8,3.029349,3.248489,0.356825,25.751392,01:23
9,3.002929,3.247998,0.357091,25.738760,01:23


<IPython.core.display.Javascript object>

In [12]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.020052,3.247675,0.356892,25.730457,01:22
1,3.075107,3.250725,0.355585,25.809040,01:23
2,3.067423,3.234122,0.359256,25.384081,01:22
3,2.990825,3.207360,0.365596,24.713757,01:23
4,2.898760,3.180631,0.373405,24.061941,01:23
5,2.812149,3.150610,0.380068,23.350315,01:23
6,2.731283,3.133968,0.386431,22.964931,01:23
7,2.649390,3.118072,0.391008,22.602766,01:23
8,2.587404,3.115484,0.393330,22.544350,01:23
9,2.568448,3.114744,0.393607,22.527657,01:23


<IPython.core.display.Javascript object>

In [13]:
learn.save_encoder("lm_enc")

<IPython.core.display.Javascript object>

In [14]:
TEXT = "A long time ago in a galaxy far, far away"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]
print("\n".join(preds))

a long time ago in a xxunk far , far away , when the time was already come , and the old days were past , i had fell into a nervous state . When i found myself musing to a stranger , i had a dream of being recalled
a long time ago in a xxunk far , far away along the south - southern fringe of Europe , a doctor had been called upon by a friend to call at his house . He was a tall , fat , brown - faced , black - haired


<IPython.core.display.Javascript object>

## Classifier

In [37]:
data_clas = DataBlock(
    blocks=(TextBlock.from_df("text", vocab=dls_lm.vocab), CategoryBlock),
    get_x=ColReader("text"),
    get_y=ColReader("label"),
    splitter=RandomSplitter(valid_pct=0.1),
)

<IPython.core.display.Javascript object>

In [38]:
bs = 96

<IPython.core.display.Javascript object>

In [39]:
dls_clas = data_clas.dataloaders(df_train, bs=bs)
dls_clas.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj and a heavy shower of rain came on , too , and odin , insulted and put to shame , had to drive with a peasant in an open cart all the seventeen versts into town . xxmaj only think now what answer could i have sent to the letter i received from you two months ago and what could i have written ? i was in despair ; i dared not write to you the truth because you would have been very unhappy , mortified and indignant , and yet what could you do ? xxmaj you could only perhaps ruin yourself , and , besides , odin would not allow it ; and fill up my letter with trifles when my heart was so full of sorrow , i could not . xxmaj for a whole month the town was full of gossip about this scandal",3
1,"xxbos “ philosophy , indeed , when all my right side is numb and i am moaning and groaning . xxmaj i ’ve tried all the medical faculty : they can xxunk beautifully , they have the whole of your disease at their xxunk , but they ’ve no idea how to cure you . xxmaj there was an enthusiastic little student here , ‘ you may die , ’ said he , ‘ but you ’ll know perfectly what disease you are dying of ! ’ xxmaj and then what a way they have sending people to specialists ! ‘ we only xxunk , ’ they say , ‘ but go to such‐and‐such a specialist , he ’ll cure you . ’ xxmaj the old doctor who used to cure all sorts of disease has completely disappeared , i assure you , now there are only specialists and they",3


<IPython.core.display.Javascript object>

In [40]:
opt_func = partial(Adam, wd=0.1)
learn = text_classifier_learner(
    dls_clas,
    AWD_LSTM,
    metrics=[accuracy],
    path=path,
    drop_mult=0.5,
    opt_func=opt_func,
    #     pretrained=False,
)

<IPython.core.display.Javascript object>

In [41]:
learn = learn.load_encoder("lm_enc")
learn = learn.to_fp16(clip=0.1)

<IPython.core.display.Javascript object>

In [42]:
learn.fit_one_cycle(2, 2e-2, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.906322,0.631312,0.772918,00:14
1,0.863009,0.588070,0.789685,00:14


<IPython.core.display.Javascript object>

In [43]:
learn.freeze_to(-2)
learn.fit_one_cycle(2, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.661431,0.559122,0.802989,00:17
1,0.565716,0.492184,0.824859,00:17


<IPython.core.display.Javascript object>

In [44]:
learn.freeze_to(-3)
learn.fit_one_cycle(2, slice(5e-3 / (2.6 ** 4), 5e-3), moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.537377,0.456716,0.837981,00:22
1,0.389472,0.420296,0.848551,00:22


<IPython.core.display.Javascript object>

In [45]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3 / (2.6 ** 4), 1e-3), moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.393879,0.413095,0.852014,00:28
1,0.343096,0.411230,0.854565,00:28


<IPython.core.display.Javascript object>

In [46]:
learn.save("clas")

Path('../input/models/clas.pth')

<IPython.core.display.Javascript object>

In [48]:
learn.export()

<IPython.core.display.Javascript object>

## Ensemble

In [67]:
# learn = text_classifier_learner(
#     dls_clas,
#     AWD_LSTM,
#     metrics=[accuracy],
#     path=path,
#     drop_mult=0.5,
# )
# learn = learn.to_fp16()
# learn.load("clas");

In [49]:
learn = load_learner(path / "export.pkl", cpu=False)

<IPython.core.display.Javascript object>

In [50]:
test_dl = learn.dls.test_dl(df_test)

<IPython.core.display.Javascript object>

In [51]:
probs, _ = learn.get_preds(dl=test_dl, reorder=False)

<IPython.core.display.Javascript object>

In [53]:
probs, probs.shape

(TensorText([[2.1218e-06, 1.5081e-05, 6.8244e-04, 9.9929e-01, 1.2332e-05],
         [1.9265e-05, 4.8878e-04, 5.9149e-03, 9.9334e-01, 2.4150e-04],
         [1.3465e-04, 2.4917e-04, 2.0104e-03, 9.9741e-01, 1.9533e-04],
         ...,
         [2.7666e-02, 9.4698e-01, 1.0632e-03, 1.2860e-02, 1.1427e-02],
         [1.8842e-02, 2.2052e-03, 4.8756e-01, 7.1737e-02, 4.1966e-01],
         [9.9670e-01, 3.0398e-04, 2.0740e-03, 7.4736e-04, 1.7761e-04]]),
 torch.Size([19617, 5]))

<IPython.core.display.Javascript object>

In [57]:
subm = pd.read_csv(path / "sample_submission.csv", index_col=0)
subm[subm.columns] = probs.numpy()
subm.head()

,0,1,2,3,4
index,,,,,
0,0.000002,0.000015,0.000682,0.999288,0.000012
1,0.000019,0.000489,0.005915,0.993336,0.000241
2,0.000135,0.000249,0.002010,0.997410,0.000195
3,0.000001,0.000007,0.000793,0.999196,0.000003
4,0.010397,0.000050,0.000986,0.000097,0.988470


<IPython.core.display.Javascript object>

In [58]:
subm.to_csv("ulmfit-fwd.csv")

<IPython.core.display.Javascript object>

In [59]:
df_train.head()

,index,text,label
0,0,"He was almost choking. There was so much, so much he wanted to say, but strange exclamations were all that came from his lips. The Pole gazed fixedly at him, at the bundle of notes in his hand; looked at odin, and was in evident perplexity.",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in perusing Jane’s last letter, and dwelling on some passages which proved that Jane had not written in spirits, when, instead of being again surprised by Mr. odin, she saw on looking up that odin was meeting her. Putting away the letter immediately and forcing a smile, she said:",1
3,3,"The captain was in the porch, keeping himself carefully out of the way of a treacherous shot, should any be intended. He turned and spoke to us, “Doctor's watch on the lookout. Dr. odin take the north side, if you please; Jim, the east; Gray, west. The watch below, all hands to load muskets. Lively, men, and careful.”",4
4,4,"“Have mercy, gentlemen!” odin flung up his hands. “Don’t write that, anyway; have some shame. Here I’ve torn my heart asunder before you, and you seize the opportunity and are fingering the wounds in both halves.... Oh, my God!”",3


<IPython.core.display.Javascript object>

In [60]:
learn.predict(df_train.iloc[0])

('3',
 TensorText(3),
 TensorText([4.3493e-05, 7.5234e-07, 4.7790e-05, 9.9980e-01, 1.1085e-04]))

<IPython.core.display.Javascript object>